In [2]:
import pandas as pd
import numpy as np
import random

s = list(range(0,30))
p = list(range(0,6))
l = list(range(0,3))

Qspl = []  #Qspl[s][p][l]  #Quantity

#creating 30x6x3 list with 0 values
for i in range(len(s)):
    temp_p = []
    for j in range(len(p)):
        temp_l = [0]*len(l)
        temp_p.append(temp_l)
    Qspl.append(temp_p)
            
#creating 30x1
Ys = [0]*len(s)  #Supplier selection binary variable, 0 or 1

Zsl = []   #l selection by supplier binary variable, 0 or 1
for i in range(len(s)):
    temp_s = [0]*len(l)
    Zsl.append(temp_s)

FCs = []
CSl = np.array([8.46, 30.16, 7.14])
CNFl = np.array([7.67, 8.08, 8.6])
CPFl = np.array([5.22, 4.09, 4.11])
CKFl = np.array([5.04, 6.35, 5.16])
CHl = np.array([4.31, 5.72, 3.15])
CPl = np.array([0.19, 0.41, 0.19])
CMl = np.array([9.4, 6.14, 5.46])
CSPl = np.array([4.07, 3.21, 2.76])
COl = np.array([7.79, 6.23, 6.84])
CONSl = np.array([1.66, 2.59, 1.44]) 
CONSFl = np.array([10.2, 10.8, 11.6])
CONNFl = np.array([5.93, 4.55, 5.0])
CONKFl = np.array([7.69, 10.18, 8])
CONHl = np.array([0.02, 0.13, 0.06])
CONPl = np.array([0.001, 0.001, 0.001])
CONMl = np.array([1.17, 0.61, 1.01])
CONSPl = np.array([0.26, 0.19, 0.32])
CONOl = np.array([6.88, 4.39, 6.22])
CLAB = 12.85
CF = 4.18
AEF = 9.817
WT = 25

Dsp = []
for i in range(len(s)):
    Dsp.append([])
    
CEHl = np.array([184.8, 178, 189.8])
CEF = 0.048
CAPs = []
DEMp = np.array([660435, 660435, 1320870, 584231, 1524080, 1371672])

In [3]:
#capacity
capacity = pd.read_excel('/Users/zaidur/Documents/MOO Corn/capacity.xlsx')
CAPs = list(capacity[" corn Production  ton"]) #capacities

In [4]:
#distance
distance = pd.read_excel('/Users/zaidur/Documents/MOO Corn/Distance.xlsx')
distance = distance.drop('Plant', axis=1)
for i in range(len(s)):
    for j in range(len(p)):
        Dsp[i].append(distance.iloc[i,j])
Dsp = np.array(Dsp)

In [5]:
fixed_cost = pd.read_excel('/Users/zaidur/Documents/MOO Corn/Fixed cost.xlsx')
FCs = list(fixed_cost['Fixed cost'])
FCs = np.array(FCs)

In [6]:
CAPs_copy = CAPs.copy()
DEMp_copy = DEMp.copy()
for plant in range(len(p)):
    distance = distance.sort_values(distance.columns[plant]) #sort the plant column by minimum distance
    quant = 0
    index = 0
    remaining_demand = DEMp[plant]
    while quant < DEMp[plant]:
        supplier = distance.index[index]
        if CAPs[supplier] >= remaining_demand:  #if capacity is larger than demand
            quant = quant + remaining_demand
            
            #assign the decision variables Qspl, Zsl, Ys
            chosen_l = random.choice([0, 1, 2])   #randomely choose one of the three harvesting style or l to be 1
            #Qspl[supplier][plant].append(remaining_demand)
            Qspl[supplier][plant][chosen_l] = remaining_demand
            if 1 not in Zsl[supplier]:    
                Zsl[supplier][chosen_l] = 1
            Ys[supplier] = 1
            
            CAPs[supplier] = CAPs[supplier] - remaining_demand
            remaining_demand = DEMp[plant] - quant
            
        else:
            quant = quant + CAPs[supplier]
            
            #Qspl[supplier][plant].append(CAPs[supplier])
            #assign the decision variables Qspl, Zsl, Ys
            chosen_l = random.choice([0, 1, 2])   #randomely choose one of the three harvesting style or l to be 1
            #Qspl[supplier][plant].append(remaining_demand)
            Qspl[supplier][plant][chosen_l] = CAPs[supplier]
            if 1 not in Zsl[supplier]:
                Zsl[supplier][chosen_l] = 1
            Ys[supplier] = 1
            
            remaining_demand = DEMp[plant] - quant
            index = index + 1
            CAPs[supplier] = 0

Qspl = np.array(Qspl)
Ys = np.array(Ys)
Zsl = np.array(Zsl)

In [7]:
ceHarvestingSum = 0   #Carbon emission harvesting, sum of CEHl * Qspl
for supplier in Qspl:
    for plant in supplier:
        ceHarvestingSum = ceHarvestingSum + CEHl@plant

In [8]:
transportSum = 0 #Carbon emission transportation, sum of DSp * Qspl
for supplier in range(len(Qspl)):
    plant_ls = []
    for plant in Qspl[supplier]:
        sum_l = sum(plant)
        plant_ls.append(sum_l)
    plant_ls = np.array(plant_ls)
    transportSum = transportSum + Dsp[supplier]@plant_ls
ceTransportSum = transportSum * CEF

In [13]:
cons1 = CSl * CONSl
csSum = 0
for supplier in Qspl:
    for plant in supplier:
        csSum = csSum + cons1@plant
csSum

185378055.4788

In [14]:
cons2 = CNFl * CONNFl
cnfSum = 0
for supplier in Qspl:
    for plant in supplier:
        cnfSum = cnfSum + cons2@plant
cnfSum

261491203.9537

In [21]:
qsplSum = 0
for supplier in Qspl:
    for plant in supplier:
        qsplSum = qsplSum + sum(plant)
clabSum = CLAB * qsplSum
clabSum

78664140.55

In [22]:
FCs

array([4309740, 3912300, 2508840, 3011850, 2384640, 2403270, 1949940,
       1974780, 1645650, 1552500, 1285470, 1620810, 1956150, 1146366,
        786186,  771282,  659502,  811026,  753894, 1097928,  489348,
        475686,  695520,  935226,  536544,  550206,  568836, 1101654,
        935226,  473202])

In [9]:
Qspl

array([[[      0,       0,       0],
        [      0,       0,       0],
        [      0,       0,       0],
        [      0,       0,       0],
        [1231000,       0,       0],
        [      0,       0,       0]],

       [[      0,       0,       0],
        [      0,       0,       0],
        [1138475,       0,       0],
        [      0,       0,       0],
        [      0,       0,       0],
        [      0,       0,       0]],

       [[      0,       0,       0],
        [      0,  587825,       0],
        [      0,       0,       0],
        [      0,       0,       0],
        [      0,       0,       0],
        [      0,       0,       0]],

       [[      0,       0,       0],
        [      0,       0,       0],
        [      0,       0,       0],
        [      0,       0,       0],
        [      0,       0,       0],
        [      0,       0,       0]],

       [[      0,       0,       0],
        [      0,   72610,       0],
        [      0,       0,    